In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from src.soporte_variables import url_atrapalo
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
res = requests.get(url_atrapalo)
print(res.status_code)
sopa = BeautifulSoup(res.content, "html.parser")
resultados = sopa.find("div", {"id": "resultados_container"})
card_results = resultados.find_all("div", {"class": "card-result"})
imagenes = [f"https://www.atrapalo.com{card.find('img', {'class': 'lazy event-img'}).get('data-original')}" if card.find("img", {'class': 'lazy event-img'}) is not None else pd.NA for card in card_results]
df_imagenes = pd.DataFrame(imagenes, columns=["imagen"])
df_imagenes.dropna(inplace=True)
ratings = [resultado.find("div", {"class": "score background-opi-very-good"}) if resultado.find("div", {"class": "score background-opi-very-good"}) is not None else pd.NA for resultado in resultados.find_all("div", {"data-qa-ta": "wrapperPhoto"})]
ratings = [rating.find("span", {"class": "opi-rating"}) if rating is not pd.NA else pd.NA for rating in ratings]
ratings = [str(rating.getText()).replace("\n", "").strip() if rating is not pd.NA else pd.NA for rating in ratings]
nombres = [str(resultado.find("span").getText()).replace("\n", "").strip() for resultado in resultados.find_all("h2", {"class": "clear nombre"})]
lugares = [resultado.find("span", {"class": "locality GATrackEvent_ubicacion show-for-small-only"}) if resultado.find("span", {"class": "locality GATrackEvent_ubicacion show-for-small-only"}) is not None else resultado.find("span") for resultado in resultados.find_all("p", {"class": "info"})]
lugares = [lugar.getText() if lugar is not None else pd.NA for lugar in lugares]
df_lugares = pd.DataFrame(lugares, columns=["lugar"]).dropna()
condicion = df_lugares["lugar"].str.startswith("2") == False
#condicion2 = df_lugares["lugar"].str.startswith("1") == False
df_lugares = df_lugares[condicion]
df_lugares = df_lugares["lugar"].str.replace("\n", "").replace("\xa0", "").replace("(Barcelona)", "").str.strip().reset_index().drop(columns=["index"])
secciones = [resultado.find("span", {"class": "type large-loc"}) for resultado in resultados.find_all("p", {"class": "info"})]
texto_secciones = [seccion.getText() if seccion is not None else pd.NA for seccion in secciones]
df_secciones = pd.DataFrame(texto_secciones, columns=["categoria"]).dropna()
df_secciones = df_secciones["categoria"].str.replace("\n", "").str.strip().reset_index().drop(columns=["index"])
df = pd.concat([pd.DataFrame(nombres, columns=["nombre_actividad"]), df_lugares, df_secciones, pd.DataFrame(ratings, columns=["puntuacion"]), df_imagenes], axis=1)
df["puntuacion"].fillna("Sin puntuación", axis=0, inplace=True)
df["imagen"].fillna("Sin imagen", axis=0, inplace=True)
df

In [ ]:
duracion = resultados.find_all("p", {"class": "info"})[::2]
duracion

In [ ]:
driver = webdriver.Chrome()
url_atrapalo = "https://www.atrapalo.com/actividades/"
driver.get(url_atrapalo)
driver.maximize_window()
sleep(1)
try:
    driver.find_element("css selector", "#soycontrolcookies > div.soycontrolcookies_header > div.soycontrolcookies_buttons > button.btn.btn-default.reject-button").click()
    sleep(1)
    driver.find_element("css selector", "#search_widget > form > input.ui-autocomplete-input").click()
    sleep(1)
    driver.find_element("css selector", "#ecommfinder_searchbox").send_keys(busqueda)
    import random
    lista_url = []
    for n in range(1, int(cantidad)+1):
        dormir = random.choice(range(1,3))
        nombre_url = driver.find_element("css selector", f"#ecommfinder_results > a:nth-child({n})").get_attribute("href")
        lista_url.append(nombre_url)
        sleep(dormir)
    driver.quit()
except:
    print("Error")
    driver.quit()

lista_url